In [5]:
from datetime import date
from Option_Classes import AmericanOption, EuropeanOption

# Create American put
american_put = AmericanOption(
    current_price=155.00,
    strike_price=170.00,
    expiry=date(2026, 5, 15),
    option_type='put',
    today_date=date(2024, 12, 1),
    interest_rate=0.045,
    sigma=0.25,
    n_steps=200
)

# Create European put for comparison
european_put = EuropeanOption(
    current_price=155.00,
    strike_price=170.00,
    expiry=date(2026, 5, 15),
    option_type='put',
    today_date=date(2024, 12, 1),
    interest_rate=0.045,
    sigma=0.25
)

# Get prices
american_price = american_put.option_price()  # Binomial tree
european_price = european_put.option_price()  # Black-Scholes

print(f"American Put: ${american_price:.4f}")
print(f"European Put: ${european_price:.4f}")
print(f"Early Exercise Premium: ${american_price - european_price:.4f}")

ModuleNotFoundError: No module named 'Option_Classes'